In [351]:
import os  
from collections import defaultdict
import string

In [350]:

import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords data if not already downloaded
nltk.download('stopwords')
# add=[',','.','-','+','!','|','<','>','?','/','&','#','@','(',')','*','^','$',';',':','{','}','[',']','_','=','|=>','<=|','=>','<=']
all_stopwords = set(stopwords.words('english')+add)

def preprocess(text):
    # Remove punctuation and make lowercase
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator).lower().split()
    
    # Remove stopwords
    text = [word for word in text if word not in all_stopwords]
    
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#to open the text file given by dataset and convert then into list and return it 

import email
def read_usenet_message(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        message = email.message_from_file(file)
        text = ''
        if message.is_multipart():
            for part in message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':
                    text += part.get_payload()
        else:
            text = message.get_payload()
        text=preprocess(text)
        return text

In [324]:
def load_data_from_folders(folder_path):
    data = {}
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        curr={}
        for txtfile in os.listdir(filepath):
            txtfilepath = os.path.join(filepath, txtfile)
            
            txt=read_usenet_message(txtfilepath)
            
            for i in txt:
                if i not in curr:
                    curr[i]=1
                else:
                    curr[i]+=1
            data[filename]=curr
            
        cnt=0
        x=data[filename].copy()
        for i in x:
            if x[i]<25:   #removing if occuring less than 5 times
                data[filename].pop(i)
        for i in data[filename]:
            cnt+=data[filename][i]
        data[filename]["total_cnt"]=cnt  #total words count
        
    return data
                
            
            


In [325]:
train_data=load_data_from_folders('Downloads/twenty+newsgroups/20_newsgroups') #the are two folders in which mini one I have used for testing and 
# storing the training data into dictionary to use when i will make the naive bayes on my own

In [326]:
def test_from_folders(folder_path): #for testing i am obtaining the word in list to calculate final probability
    xdata = []
    ydata=[]
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        curr=[]
        for txtfile in os.listdir(filepath):
            txtfilepath = os.path.join(filepath, txtfile)
            
            txt=read_usenet_message(txtfilepath)
            
            for i in txt:
                curr.append(i)
            xdata.append(curr)
            ydata.append(filename)
            
        
    return xdata,ydata

In [327]:
x_test,y_test=test_from_folders('Downloads/twenty+newsgroups/mini_newsgroups')

In [331]:
#testing function
import numpy as np
def test(txt):
    pred_prob={}
    for i in train_data:
        pred_prob[i]=0.00     #initialising all with 0
        val=0.00
        for j in txt:
            if j not in train_data[i]:
                val+=np.log10(((1.0  )/(train_data[i]["total_cnt"]*1.0 ))) #laplace approximation
            else:
                val+=np.log10(((train_data[i][j]*1.0  )/(train_data[i]["total_cnt"]*1.0 )))
                # print(train_data[i]["total_cnt"])
            pred_prob[i]=val
            # print(pred_prob[i])
       
    mx=-1e100
    ans="abc"
    for i in pred_prob:
        # print(i,pred_prob[i])
        if(pred_prob[i]>mx):
            mx=pred_prob[i]
            ans=i
    return ans


In [337]:
y_pred=[]
for i in x_test:
    y_pred.append(test(i))
    
# y_pred

In [338]:
y_pred

['comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windo

In [344]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)  #it is giving 100% correct idk why may be it is overfitted or what

array([[100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 100,   0,   0,  

INBUILT SKLEARN NAIVE BAYES

In [345]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB

# Your dictionary containing word counts for each class label
data = train_data.copy()
for i in data:
    data[i].pop("total_cnt")
    
# Extracting features (unique words) from previous build x_data dictionary so that we can firt into sklearn MultinomialNB
all_words = set(word for class_dict in data.values() for word in class_dict.keys())

# Initialize a DictVectorizer
vectorizer = DictVectorizer(sparse=False)

# Transform your dictionaries into feature matrices
X = vectorizer.fit_transform([{word: class_dict.get(word, 0) for word in all_words} for class_dict in data.values()])
y = list(data.keys())

clf = MultinomialNB()
clf.fit(X, y)


MultinomialNB()

In [348]:
y_pred_sk=[]
# Convert list of lists into list of dictionaries of word counts
x_test_counts = []
for text_list in x_test:
    word_count = {}
    for word in text_list:
        word_count[word] = word_count.get(word, 0) + 1
    x_test_counts.append(word_count)

# Vectorize test data
X_test = vectorizer.transform([{word: word_count.get(word, 0) for word in all_words} for word_count in x_test_counts])

predictions = clf.predict(X_test)
for i in predictions:  
    y_pred_sk.append(i)


In [349]:
confusion_matrix(y_test,y_pred_sk)

array([[100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 100,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 100,   0,   0,  